In [1]:
!pip install imbalanced-learn

     |████████████████████████████████| 258 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.5 MB 62.1 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from imblearn.pipeline import Pipeline

## Подготовка

In [6]:
df = df.drop('Unnamed: 0', axis=1)

In [7]:
print(df['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64


In [8]:
# Ограничение данных до 3500 комментариев
df = df.sample(n=10000, random_state=42)

In [9]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stopwords_eng = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
# Функция для определения POS-тега для лемматизации
def get_wordnet_pos(word):
    """Возвращает тег POS для лемматизации"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()


In [11]:
# Лемматизация текста с POS-тегами
def lemmatize_text(text):
    words = word_tokenize(text)  # Разбиваем текст на слова
    return ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words])


In [12]:
# Применение лемматизации
df['lemmatized_text'] = df['text'].apply(lemmatize_text)

# Проверка результата
print(df['lemmatized_text'].head())

31015     Sometime back , I just happen to log on to www...
102832    `` The late edit be much well , do n't make th...
67317     `` October 2007 ( UTC ) I would think you 'd b...
81091     Thanks for the tip on the currency translation...
90091     I would argue that if content on the Con in co...
Name: lemmatized_text, dtype: object


## Обучение

In [13]:
# Разделение данных
X_train_text, X_temp_text, y_train, y_temp = train_test_split(df['lemmatized_text'], df['toxic'], test_size=0.3, random_state=42)
X_val_text, X_test_text, y_val, y_test = train_test_split(X_temp_text, y_temp, test_size=0.5, random_state=42)

In [14]:
# TF-IDF
tfidf = TfidfVectorizer(stop_words=list(stopwords_eng), max_features=10000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(X_train_text)
X_val = tfidf.transform(X_val_text)
X_test = tfidf.transform(X_test_text)

In [ ]:
# Logistic Regression с кросс-валидацией
param_grid_lr = {'C': [0.7, 1.0, 1.5, 2.0], 'penalty': ['l1']}
grid_lr = GridSearchCV(LogisticRegression(max_iter=10000, class_weight='balanced', solver='liblinear'), param_grid_lr, scoring='f1', cv=5)
grid_lr.fit(X_train, y_train)
best_model_lr = grid_lr.best_estimator_


In [ ]:
# Оценка на валидационной выборке для Logistic Regression
y_val_pred_proba_lr = best_model_lr.predict_proba(X_val)[:, 1]
y_val_pred_lr = (y_val_pred_proba_lr > 0.79).astype(int)
f1_val_lr = f1_score(y_val, y_val_pred_lr)
print(f'F1 для Logistic Regression на валидационной выборке: {f1_val_lr:.4f}')

In [ ]:
best_threshold = 0
best_f1 = 0
y_pred_proba = best_model_lr.predict_proba(X_test)[:, 1]
for threshold in [x * 0.01 for x in range(70, 81)]:
    y_pred_lr = (y_pred_proba > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_lr)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'Лучший порог: {best_threshold}, F1: {best_f1:.4f}')


In [ ]:
# RandomForest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_val_pred_rf = model_rf.predict(X_val)
f1_val_rf = f1_score(y_val, y_val_pred_rf)
print(f'F1 для RandomForest на валидационной выборке: {f1_val_rf:.4f}')

In [ ]:
# CatBoost
model_cat = CatBoostClassifier(iterations=500, depth=6, task_type="CPU", verbose=100)
model_cat.fit(X_train, y_train)
y_val_pred_cat = model_cat.predict(X_val)
f1_val_cat = f1_score(y_val, y_val_pred_cat)
print(f'F1 для CatBoost на валидационной выборке: {f1_val_cat:.4f}')

## Выводы

In [ ]:
# Выбор лучшей модели
best_model = None
best_f1 = 0

if f1_val_lr > best_f1:
    best_model = best_model_lr
    best_f1 = f1_val_lr

if f1_val_cat > best_f1:
    best_model = model_cat
    best_f1 = f1_val_cat

if f1_val_rf > best_f1:
    best_model = model_rf
    best_f1 = f1_val_rf

print(f'Лучшая модель: {best_model.__class__.__name__} с F1-метрикой: {best_f1:.4f} на валидации')

# Оценка на тестовой выборке
if best_model.__class__.__name__ == 'LogisticRegression':
    y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]
    y_test_pred = (y_test_pred_proba > best_threshold).astype(int)
else:
    y_test_pred = best_model.predict(X_test)

f1_test = f1_score(y_test, y_test_pred)
print(f'F1-метрика для лучшей модели ({best_model.__class__.__name__}) на тестовой выборке: {f1_test:.4f}')

Вывод:

Проведена необходимая предобработка датасета, включающая в себя лемматизацию и кодирование методом tf-idf, поскольку этот метод кодирования показал себя лучше, чем мешок слов. Дисбаланас классов был обнаружен, но не был устранен поскольку эксперименты показали, что он не влияет на метрику негативно. Также модели:логстическая регрессия, случайный лес,Catboost сравнивались на валидационной выборке. Лучшей оказалась логистическая регрессия, которая на тестовой показала f1=0.76, что больше принятого порога 0.75